In [ ]:
import jax
import jax.numpy as jnp
import jax.nn as jnn
from jax import random, vmap, jit, lax
from functools import partial
import numpy as np
import pandas as pd
import os
import time
import pickle
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

print("="*70)
print("JAX RANDOM FOREST FOR NAO EEG CONTROL")
print("="*70)
print(f"JAX version: {jax.__version__}")
print(f"Device: {jax.default_backend().upper()}")
print(f"Author: Yash272001")
print("="*70)

# Configuration
BASE_DATA_PATH = r"C:\Users\yaskk\JAX Random Forest NAO Control\Professor Data\data\data"
OUTPUT_PATH = os.path.join(BASE_DATA_PATH, 'output')
PLOTS_PATH = os.path.join(OUTPUT_PATH, 'plots')

N_TREES = 40
MAX_DEPTH = 8
MIN_SAMPLES_SPLIT = 100
MIN_SAMPLES_LEAF = 50
BOOTSTRAP_RATIO = 0.6
FEATURE_SUBSAMPLE_RATIO = 0.2
N_THRESHOLD_SAMPLES = 4
SEED = 42
DESIRED_ROWS = 160

jax.config.update("jax_enable_x64", False)

# Set matplotlib style with fallback
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('default')

# ==================================================
# DATA PROCESSING
# ==================================================

def normalize_class_label(label):
    """Normalize class labels to standard format"""
    label_lower = label.lower()
    if 'backward' in label_lower:
        return 'backward'
    elif 'forward' in label_lower:
        return 'forward'
    elif 'landing' in label_lower:
        return 'landing'
    elif 'left' in label_lower:
        return 'left'
    elif 'right' in label_lower:
        return 'right'
    elif 'take' in label_lower or 'takeoff' in label_lower:
        return 'takeoff'
    return label

def read_csv_flexible(file_path):
    """Try multiple separators to read CSV"""
    for sep in [',', '\t', r'\s+']:
        try:
            df = pd.read_csv(file_path, sep=sep)
            if not df.empty and df.shape[1] > 1:
                return df
        except:
            continue
    try:
        return pd.read_csv(file_path, header=None)
    except:
        raise ValueError(f"Could not parse CSV: {file_path}")

def load_and_process_data(base_path, desired_rows):
    """Load and process EEG data from CSV files"""
    print("\nLoading EEG data...")
    
    all_samples = []
    all_labels = []
    successful_files = 0
    
    for root, dirs, files in os.walk(base_path):
        if 'output' in root:
            continue
        csv_files = [f for f in files if f.endswith('.csv')]
        if csv_files:
            class_label_raw = os.path.basename(root)
            if class_label_raw.startswith(('group', 'individual', 'Test')):
                continue
            class_label = normalize_class_label(class_label_raw)
            
            for csv_file in csv_files:
                try:
                    file_path = os.path.join(root, csv_file)
                    df = read_csv_flexible(file_path)
                    if df.empty:
                        continue
                    numeric_cols = df.select_dtypes(include=[np.number]).columns
                    if len(numeric_cols) == 0:
                        continue
                    df = df[numeric_cols]
                    
                    if len(df) < desired_rows:
                        padding = pd.DataFrame(0, index=range(desired_rows - len(df)), 
                                             columns=df.columns)
                        df = pd.concat([df, padding], ignore_index=True)
                    elif len(df) > desired_rows:
                        df = df.iloc[:desired_rows]
                    
                    df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
                    
                    for idx, row in df.iterrows():
                        all_samples.append(row.values.astype(np.float32))
                        all_labels.append(class_label)
                    
                    successful_files += 1
                    if successful_files % 200 == 0:
                        print(f"  Processed {successful_files} files...")
                except:
                    pass
    
    print(f"Loaded {successful_files} files, {len(all_samples):,} samples")
    
    max_features = max(len(s) for s in all_samples)
    X_list = []
    for sample in all_samples:
        if len(sample) < max_features:
            padded = np.zeros(max_features, dtype=np.float32)
            padded[:len(sample)] = sample
            X_list.append(padded)
        else:
            X_list.append(sample[:max_features])
    
    X = np.array(X_list, dtype=np.float32)
    y = np.array(all_labels)
    
    class_names = sorted(np.unique(y))
    label_to_idx = {label: idx for idx, label in enumerate(class_names)}
    y_encoded = np.array([label_to_idx[label] for label in y], dtype=np.int32)
    
    print(f"Dataset shape: X={X.shape}, y={y_encoded.shape}")
    print(f"Classes: {class_names}\n")
    
    return X, y_encoded, class_names, label_to_idx

# ==================================================
# JAX FUNCTIONS
# ==================================================

@partial(jit, static_argnums=(3, 4))
def compute_weighted_gini_jax(X_col, y, threshold, n_classes, min_leaf):
    """Calculate weighted Gini impurity for a split"""
    left_mask = X_col <= threshold
    n_total = X_col.shape[0]
    n_left = jnp.sum(left_mask)
    n_right = n_total - n_left
    
    too_small = (n_left < min_leaf) | (n_right < min_leaf)
    penalty = jnp.where(too_small, 1e9, 0.0)
    
    def safe_gini(mask, labels):
        n = jnp.sum(mask)
        safe_n = jnp.maximum(n, 1)
        onehot = jnn.one_hot(labels, n_classes)
        mask_expanded = mask[:, jnp.newaxis]
        masked_onehot = onehot * mask_expanded
        counts = jnp.sum(masked_onehot, axis=0)
        probs = counts / safe_n
        gini = 1.0 - jnp.sum(probs * probs)
        return jnp.where(n > 0, gini, 1.0)
    
    left_gini = safe_gini(left_mask, y)
    right_gini = safe_gini(jnp.logical_not(left_mask), y)
    
    weighted = (n_left / n_total) * left_gini + (n_right / n_total) * right_gini
    
    return weighted + penalty

def find_best_split_jax(X, y, n_classes, min_leaf, key):
    """Find best feature and threshold for splitting using JAX"""
    n_samples, n_features = X.shape
    n_try = max(1, int(n_features * FEATURE_SUBSAMPLE_RATIO))
    
    X_jax = jnp.array(X)
    y_jax = jnp.array(y)
    
    key, subkey = random.split(key)
    feats = random.choice(subkey, n_features, (n_try,), replace=False)
    
    def best_for_feat(fidx):
        col = X_jax[:, fidx]
        sorted_col = jnp.sort(col)
        step = jnp.maximum(1, n_samples // N_THRESHOLD_SAMPLES)
        thr_ix = jnp.arange(step, n_samples, step)[:N_THRESHOLD_SAMPLES]
        thrs = jnp.take(sorted_col, thr_ix)
        
        imp = vmap(lambda t: compute_weighted_gini_jax(col, y_jax, t, n_classes, min_leaf))(thrs)
        
        k = jnp.argmin(imp)
        return imp[k], thrs[k], fidx
    
    imps, thrs, fids = vmap(best_for_feat)(feats)
    
    k = jnp.argmin(imps)
    return int(fids[k]), float(thrs[k]), float(imps[k])

# ==================================================
# DECISION TREE
# ==================================================

class JAXDecisionTreeParams:
    """Decision tree structure storage"""
    
    def __init__(self):
        self.nodes = []
    
    def add_leaf(self, class_label, samples):
        node_id = len(self.nodes)
        self.nodes.append({
            'is_leaf': True,
            'class': int(class_label),
            'samples': int(samples),
            'node_id': node_id,
            'left': node_id,
            'right': node_id
        })
        return node_id
    
    def add_split(self, feature, threshold, left, right, samples):
        node_id = len(self.nodes)
        self.nodes.append({
            'is_leaf': False,
            'feature': int(feature),
            'threshold': float(threshold),
            'left': left,
            'right': right,
            'samples': int(samples),
            'node_id': node_id
        })
        return node_id

def build_jax_tree(X, y, n_classes, max_depth, key):
    """Build decision tree using JAX-accelerated split finding"""
    tree_params = JAXDecisionTreeParams()
    
    def build_node(X_node, y_node, depth, node_key):
        n_samples = X_node.shape[0]
        
        if depth >= max_depth or n_samples < MIN_SAMPLES_SPLIT:
            unique_labels, counts = np.unique(y_node, return_counts=True)
            majority = unique_labels[np.argmax(counts)]
            return tree_params.add_leaf(majority, n_samples)
        
        if len(np.unique(y_node)) == 1:
            return tree_params.add_leaf(y_node[0], n_samples)
        
        node_key, split_key = random.split(node_key)
        best_feat, threshold, impurity = find_best_split_jax(
            X_node, y_node, n_classes, MIN_SAMPLES_LEAF, split_key
        )
        
        if float(impurity) >= 1e8:
            unique_labels, counts = np.unique(y_node, return_counts=True)
            majority = unique_labels[np.argmax(counts)]
            return tree_params.add_leaf(majority, n_samples)
        
        split_mask = X_node[:, best_feat] <= threshold
        X_left = X_node[split_mask]
        y_left = y_node[split_mask]
        X_right = X_node[np.logical_not(split_mask)]
        y_right = y_node[np.logical_not(split_mask)]
        
        node_key, left_key, right_key = random.split(node_key, 3)
        left_child = build_node(X_left, y_left, depth + 1, left_key)
        right_child = build_node(X_right, y_right, depth + 1, right_key)
        
        return tree_params.add_split(best_feat, threshold, left_child, right_child, n_samples)
    
    root_id = build_node(np.array(X), np.array(y), 0, key)
    return tree_params, root_id

@partial(jit, static_argnums=(2,))
def predict_single_jax(tree_arrays, x, root_id):
    """JIT-compiled prediction for single sample"""
    nodes, features, thresholds, left, right, classes = tree_arrays
    
    def cond(carry):
        node_idx, _ = carry
        return nodes[node_idx] == 0
    
    def body(carry):
        node_idx, _ = carry
        goes_left = x[features[node_idx]] <= thresholds[node_idx]
        next_idx = jnp.where(goes_left, left[node_idx], right[node_idx])
        return (next_idx, 0)
    
    node_idx, _ = lax.while_loop(cond, body, (root_id, 0))
    
    return classes[node_idx]

def tree_to_arrays(tree_params):
    """Convert tree to JAX arrays for efficient prediction"""
    n_nodes = len(tree_params.nodes)
    nodes = np.zeros(n_nodes, dtype=np.int32)
    features = np.zeros(n_nodes, dtype=np.int32)
    thresholds = np.zeros(n_nodes, dtype=np.float32)
    left = np.zeros(n_nodes, dtype=np.int32)
    right = np.zeros(n_nodes, dtype=np.int32)
    classes = np.zeros(n_nodes, dtype=np.int32)
    
    for i, node in enumerate(tree_params.nodes):
        if node['is_leaf']:
            nodes[i] = 1
            classes[i] = node['class']
            left[i] = node['left']
            right[i] = node['right']
        else:
            features[i] = node['feature']
            thresholds[i] = node['threshold']
            left[i] = node['left']
            right[i] = node['right']
    
    return (jnp.array(nodes), jnp.array(features), jnp.array(thresholds),
            jnp.array(left), jnp.array(right), jnp.array(classes))

# ==================================================
# RANDOM FOREST
# ==================================================

class JAXRandomForest:
    """Random Forest implemented in pure JAX"""
    
    def __init__(self, n_trees=40, max_depth=8, bootstrap_ratio=0.6, seed=42):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.bootstrap_ratio = bootstrap_ratio
        self.seed = seed
        self.trees = []
        self.n_classes = None
        self.train_scores = []
        self.val_scores = []
        self.train_losses = []
        self.val_losses = []
    
    def fit(self, X_train, y_train, X_val=None, y_val=None, verbose=True):
        """Fit random forest on training data"""
        self.n_classes = len(np.unique(y_train))
        n_samples = X_train.shape[0]
        n_bootstrap = int(n_samples * self.bootstrap_ratio)
        
        X_train_jax = jnp.array(X_train)
        y_train_jax = jnp.array(y_train)
        
        if verbose:
            print("Training Random Forest")
            print(f"Trees: {self.n_trees}, Max depth: {self.max_depth}")
            print(f"Bootstrap: {self.bootstrap_ratio}, Feature subsample: {FEATURE_SUBSAMPLE_RATIO}\n")
        
        key = random.PRNGKey(self.seed)
        track_interval = 10
        track_points = list(range(1, self.n_trees + 1, track_interval)) + [self.n_trees]
        track_points = sorted(set(track_points))
        
        for i in range(self.n_trees):
            if verbose and ((i + 1) % 10 == 0 or i == 0):
                print(f"Tree {i + 1}/{self.n_trees}...")
            
            key, boot_key, tree_key = random.split(key, 3)
            indices = random.choice(boot_key, n_samples, (n_bootstrap,), replace=True)
            
            X_boot = X_train_jax[indices]
            y_boot = y_train_jax[indices]
            
            X_boot_np = np.array(X_boot)
            y_boot_np = np.array(y_boot)
            
            tree_params, root_id = build_jax_tree(
                X_boot_np, y_boot_np, self.n_classes, self.max_depth, tree_key
            )
            
            tree_arrays = tree_to_arrays(tree_params)
            self.trees.append({
                'params': tree_params,
                'root': root_id,
                'arrays': tree_arrays
            })
            
            if (i + 1) in track_points:
                train_acc = self.score(X_train, y_train)
                self.train_scores.append(train_acc)
                self.train_losses.append(1.0 - train_acc)
                if X_val is not None:
                    val_acc = self.score(X_val, y_val)
                    self.val_scores.append(val_acc)
                    self.val_losses.append(1.0 - val_acc)
                    if verbose:
                        print(f"   Train: {train_acc*100:.2f}% | Val: {val_acc*100:.2f}%")
        
        if verbose:
            print("\nTraining complete\n")
        return self
    
    def predict(self, X):
        """Predict class labels using majority voting"""
        n_samples = X.shape[0]
        all_preds = np.zeros((n_samples, len(self.trees)), dtype=np.int32)
        X_jax = jnp.array(X)
        
        for tree_idx, tree in enumerate(self.trees):
            pred_fn = lambda x: predict_single_jax(tree['arrays'], x, tree['root'])
            preds = vmap(pred_fn)(X_jax)
            all_preds[:, tree_idx] = np.array(preds)
        
        final_preds = []
        for i in range(n_samples):
            values, counts = np.unique(all_preds[i], return_counts=True)
            final_preds.append(values[np.argmax(counts)])
        return np.array(final_preds, dtype=np.int32)
    
    def score(self, X, y):
        """Calculate accuracy"""
        return float(np.mean(self.predict(X) == y))

# ==================================================
# VISUALIZATION
# ==================================================

def plot_per_class_metrics(y_true, y_pred, class_names, output_path):
    """Plot per-class performance metrics (bar chart)"""
    print("Generating per-class metrics plot...")
    os.makedirs(output_path, exist_ok=True)
    
    n_classes = len(class_names)
    precision = np.zeros(n_classes, dtype=np.float32)
    recall = np.zeros(n_classes, dtype=np.float32)
    f1 = np.zeros(n_classes, dtype=np.float32)
    
    for class_idx in range(n_classes):
        mask_true = y_true == class_idx
        mask_pred = y_pred == class_idx
        tp = np.sum(mask_true & mask_pred)
        fp = np.sum(np.logical_not(mask_true) & mask_pred)
        fn = np.sum(mask_true & np.logical_not(mask_pred))
        
        precision[class_idx] = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall[class_idx] = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1[class_idx] = 2 * precision[class_idx] * recall[class_idx] / (precision[class_idx] + recall[class_idx]) if (precision[class_idx] + recall[class_idx]) > 0 else 0
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    x = np.arange(n_classes)
    width = 0.25
    
    bars1 = ax.bar(x - width, precision, width, label='Precision', color='#5DA5DA', alpha=0.9)
    bars2 = ax.bar(x, recall, width, label='Recall', color='#60BD68', alpha=0.9)
    bars3 = ax.bar(x + width, f1, width, label='F1-Score', color='#F17CB0', alpha=0.9)
    
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}',
                   ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    ax.set_xlabel('Classes', fontsize=14, fontweight='bold')
    ax.set_ylabel('Score', fontsize=14, fontweight='bold')
    ax.set_title('Per-Class Performance Metrics', fontsize=16, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(class_names, rotation=0, ha='center')
    ax.legend(fontsize=12)
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 1.1)
    
    plt.tight_layout()
    metrics_path = os.path.join(output_path, 'per_class_metrics.png')
    plt.savefig(metrics_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved: {metrics_path}")

def plot_confusion_matrix_enhanced(y_true, y_pred, class_names, output_path):
    """Plot enhanced confusion matrix with percentages"""
    print("Generating enhanced confusion matrix...")
    os.makedirs(output_path, exist_ok=True)
    
    n_classes = len(class_names)
    cm = np.zeros((n_classes, n_classes), dtype=np.int32)
    for i in range(len(y_true)):
        cm[y_true[i], y_pred[i]] += 1
    
    fig, ax = plt.subplots(figsize=(14, 12))
    
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    for i in range(n_classes):
        for j in range(n_classes):
            if i == j:
                color = plt.cm.Greens(cm_norm[i, j])
            else:
                color = plt.cm.Reds(cm_norm[i, j] * 0.8)
            ax.add_patch(plt.Rectangle((j-0.5, i-0.5), 1, 1, facecolor=color, edgecolor='white', linewidth=2))
            
            text_color = 'white' if (i == j and cm_norm[i, j] > 0.5) or (i != j and cm_norm[i, j] > 0.4) else 'black'
            ax.text(j, i, f'{cm[i, j]}\n({cm_norm[i, j]*100:.1f}%)',
                   ha='center', va='center', fontsize=11, fontweight='bold', color=text_color)
    
    ax.set_xlim(-0.5, n_classes-0.5)
    ax.set_ylim(n_classes-0.5, -0.5)
    ax.set_xticks(range(n_classes))
    ax.set_yticks(range(n_classes))
    ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=12)
    ax.set_yticklabels(class_names, fontsize=12)
    ax.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
    ax.set_ylabel('True Label', fontsize=14, fontweight='bold')
    ax.set_title('Confusion Matrix - JAX Random Forest\nNAO Robot EEG Control', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    cm_path = os.path.join(output_path, 'confusion_matrix_enhanced.png')
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved: {cm_path}")
    
    return cm

def plot_training_curves_dual(model, output_path):
    """Plot training loss and accuracy over time (dual plot)"""
    print("Generating training curves (dual plot)...")
    os.makedirs(output_path, exist_ok=True)
    
    if not model.train_scores:
        return
    
    n_trees_tracked = len(model.train_scores)
    track_interval = model.n_trees // n_trees_tracked if n_trees_tracked > 1 else 1
    epochs = [i * track_interval for i in range(1, n_trees_tracked + 1)]
    
    train_loss = model.train_losses
    train_acc = [s * 100 for s in model.train_scores]
    test_acc = [s * 100 for s in model.val_scores] if model.val_scores else None
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))
    
    ax1.plot(epochs, train_loss, 'o-', linewidth=2, markersize=4, color='#1f77b4', label='Training Loss')
    ax1.set_xlabel('Trees', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Loss', fontsize=13, fontweight='bold')
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold')
    ax1.legend(fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(epochs, train_acc, 'o-', linewidth=2, markersize=4, 
             color='#2ca02c', label='Training Accuracy')
    if test_acc:
        ax2.plot(epochs, test_acc, 's-', linewidth=2, markersize=4, 
                color='#d62728', label='Validation Accuracy')
    ax2.set_xlabel('Trees', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Accuracy (%)', fontsize=13, fontweight='bold')
    ax2.set_title('Accuracy Over Time', fontsize=15, fontweight='bold')
    ax2.legend(fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    curves_path = os.path.join(output_path, 'training_curves_dual.png')
    plt.savefig(curves_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved: {curves_path}")

def plot_training_history_comprehensive(model, output_path):
    """Plot comprehensive training history (4-panel layout)"""
    print("Generating comprehensive training history...")
    os.makedirs(output_path, exist_ok=True)
    
    if not model.train_scores:
        return
    
    n_trees_tracked = len(model.train_scores)
    track_interval = model.n_trees // n_trees_tracked if n_trees_tracked > 1 else 1
    epochs = [i * track_interval for i in range(1, n_trees_tracked + 1)]
    
    train_loss = model.train_losses
    train_acc = [s * 100 for s in model.train_scores]
    test_acc = [s * 100 for s in model.val_scores] if model.val_scores else None
    
    fig = plt.figure(figsize=(16, 12))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    
    ax_main = fig.add_subplot(gs[0, :])
    ax1 = ax_main
    ax2 = ax1.twinx()
    
    line1 = ax1.plot(epochs, train_loss, 'o-', linewidth=2, markersize=3, 
                     color='#1f77b4', label='Training Loss')
    line2 = ax2.plot(epochs, train_acc, 's-', linewidth=2, markersize=3, 
                     color='#2ca02c', label='Training Accuracy')
    if test_acc:
        line3 = ax2.plot(epochs, test_acc, '^-', linewidth=2, markersize=3, 
                        color='#d62728', label='Validation Accuracy')
    
    ax1.set_xlabel('Trees', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Loss', fontsize=13, fontweight='bold', color='#1f77b4')
    ax2.set_ylabel('Accuracy (%)', fontsize=13, fontweight='bold', color='#2ca02c')
    ax1.tick_params(axis='y', labelcolor='#1f77b4')
    ax2.tick_params(axis='y', labelcolor='#2ca02c')
    ax1.set_title('Training History - NAO Robot EEG Control\n\nLoss and Accuracy', 
                 fontsize=15, fontweight='bold')
    
    lines = line1 + line2 + (line3 if test_acc else [])
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='upper right', fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    ax3 = fig.add_subplot(gs[1, 0])
    ax3.plot(epochs, train_loss, 'o-', linewidth=2, markersize=3, color='#1f77b4')
    ax3.fill_between(epochs, train_loss, alpha=0.3, color='#1f77b4')
    ax3.set_xlabel('Trees', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Training Loss', fontsize=12, fontweight='bold')
    ax3.set_title('Training Loss Trend', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.plot(epochs, train_acc, 's-', linewidth=2, markersize=3, 
            color='#2ca02c', label='Train')
    ax4.fill_between(epochs, train_acc, alpha=0.3, color='#2ca02c')
    if test_acc:
        ax4.plot(epochs, test_acc, '^-', linewidth=2, markersize=3, 
                color='#d62728', label='Validation')
        ax4.fill_between(epochs, test_acc, alpha=0.3, color='#d62728')
    ax4.set_xlabel('Trees', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
    ax4.set_title('Accuracy Comparison', fontsize=14, fontweight='bold')
    ax4.legend(fontsize=11)
    ax4.grid(True, alpha=0.3)
    
    ax5 = fig.add_subplot(gs[2, :])
    ax5.axis('off')
    
    final_train_acc = model.train_scores[-1] * 100
    final_test_acc = model.val_scores[-1] * 100 if model.val_scores else 0
    final_train_loss = train_loss[-1]
    
    summary_text = f"""
    TRAINING SUMMARY
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    Total Trees: {model.n_trees}
    Final Training Accuracy: {final_train_acc:.2f}%
    Final Validation Accuracy: {final_test_acc:.2f}%
    Final Training Loss: {final_train_loss:.4f}
    Best Training Accuracy: {max(model.train_scores)*100:.2f}%
    Best Validation Accuracy: {max(model.val_scores)*100 if model.val_scores else 0:.2f}%
    """
    
    ax5.text(0.5, 0.5, summary_text, transform=ax5.transAxes,
            fontsize=12, verticalalignment='center', horizontalalignment='center',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3),
            family='monospace')
    
    history_path = os.path.join(output_path, 'training_history_comprehensive.png')
    plt.savefig(history_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved: {history_path}")

def print_classification_report(y_true, y_pred, class_names):
    """Print classification metrics"""
    print("\n" + "=" * 70)
    print("CLASSIFICATION REPORT")
    print("=" * 70)
    print(f"\n{'Class':<12} {'Precision':<12} {'Recall':<12} {'F1':<12} {'Support':<12}")
    print("-" * 70)
    
    for class_idx, class_name in enumerate(class_names):
        mask_true = y_true == class_idx
        mask_pred = y_pred == class_idx
        tp = np.sum(mask_true & mask_pred)
        fp = np.sum(np.logical_not(mask_true) & mask_pred)
        fn = np.sum(mask_true & np.logical_not(mask_pred))
        
        prec = tp / (tp + fp) if (tp + fp) > 0 else 0
        rec = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
        supp = np.sum(mask_true)
        
        print(f"{class_name:<12} {prec:>11.4f} {rec:>11.4f} {f1:>11.4f} {int(supp):>11}")
    
    accuracy = np.mean(y_true == y_pred)
    print("-" * 70)
    print(f"{'Overall':<12} {'':<12} {'':<12} {'':<12} {len(y_true):>11}")
    print(f"{'Accuracy':<12} {accuracy:>11.4f}")
    print("=" * 70)

# ==================================================
# MAIN
# ==================================================

def main():
    """Main execution"""
    try:
        os.makedirs(OUTPUT_PATH, exist_ok=True)
        os.makedirs(PLOTS_PATH, exist_ok=True)
        
        X, y, class_names, label_to_idx = load_and_process_data(BASE_DATA_PATH, DESIRED_ROWS)
        
        print("Splitting data...")
        n_samples = X.shape[0]
        n_test = int(n_samples * 0.2)
        n_val = int(n_samples * 0.1)
        
        np.random.seed(SEED)
        indices = np.random.permutation(n_samples)
        train_idx = indices[:n_samples - n_test - n_val]
        val_idx = indices[n_samples - n_test - n_val:n_samples - n_test]
        test_idx = indices[n_samples - n_test:]
        
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        
        print(f"Train: {len(X_train):,} | Val: {len(X_val):,} | Test: {len(X_test):,}\n")
        
        start_time = time.time()
        model = JAXRandomForest(
            n_trees=N_TREES, 
            max_depth=MAX_DEPTH, 
            bootstrap_ratio=BOOTSTRAP_RATIO, 
            seed=SEED
        )
        model.fit(X_train, y_train, X_val, y_val, verbose=True)
        training_time = time.time() - start_time
        
        print("="*70)
        print("EVALUATION")
        print("="*70)
        test_acc = model.score(X_test, y_test)
        
        print(f"Training time: {training_time:.1f}s ({training_time/60:.1f} min)")
        print(f"Test accuracy: {test_acc*100:.2f}%")
        print(f"NAO control success rate: {test_acc*100:.2f}%\n")
        
        y_pred = model.predict(X_test)
        
        plot_per_class_metrics(y_test, y_pred, class_names, PLOTS_PATH)
        cm = plot_confusion_matrix_enhanced(y_test, y_pred, class_names, PLOTS_PATH)
        plot_training_curves_dual(model, PLOTS_PATH)
        plot_training_history_comprehensive(model, PLOTS_PATH)
        
        print_classification_report(y_test, y_pred, class_names)
        
        model_data = {
            'model': model,
            'class_names': class_names,
            'label_to_idx': label_to_idx,
            'test_accuracy': test_acc,
            'confusion_matrix': cm,
            'training_time': training_time,
            'author': 'Yash272001',
            'date': '2025-01-19',
            'jax_version': jax.__version__
        }
        
        model_path = os.path.join(OUTPUT_PATH, 'jax_random_forest_model.pkl')
        with open(model_path, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"\nModel saved: {model_path}")
        
        print("\n" + "="*70)
        print("COMPLETE")
        print("="*70)
        print("Generated plots:")
        print("  1. per_class_metrics.png")
        print("  2. confusion_matrix_enhanced.png")
        print("  3. training_curves_dual.png")
        print("  4. training_history_comprehensive.png")
        print("="*70)
        
    except Exception as e:
        print(f"\nError: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

JAX RANDOM FOREST FOR NAO EEG CONTROL
JAX version: 0.7.2
Device: CPU
Author: Yash272001

Loading EEG data...
  Processed 200 files...
  Processed 400 files...
  Processed 600 files...
  Processed 800 files...
  Processed 1000 files...
  Processed 1200 files...
Loaded 1203 files, 192,480 samples
Dataset shape: X=(192480, 32), y=(192480,)
Classes: [np.str_('backward'), np.str_('forward'), np.str_('landing'), np.str_('left'), np.str_('right'), np.str_('takeoff')]

Splitting data...
Train: 134,736 | Val: 19,248 | Test: 38,496

Training Random Forest
Trees: 40, Max depth: 8
Bootstrap: 0.6, Feature subsample: 0.2

Tree 1/40...
   Train: 71.60% | Val: 71.28%
Tree 10/40...
   Train: 93.08% | Val: 93.12%
Tree 20/40...
   Train: 93.79% | Val: 93.71%
Tree 30/40...
   Train: 94.34% | Val: 94.25%
Tree 40/40...
   Train: 94.79% | Val: 94.72%

Training complete

EVALUATION
Training time: 1549.7s (25.8 min)
Test accuracy: 94.36%
NAO control success rate: 94.36%

Generating per-class metrics plot...
Sa